In [1]:
from modelscope import AutoModelForCausalLM, AutoTokenizer, snapshot_download
#target_model="qwen/Qwen-7B-Chat"
target_model="qwen/Qwen-14B-Chat"
llm_weight_workspace='/mnt/llm-data/'
model_name_or_path=llm_weight_workspace+target_model
llm_finetune_checkpoint_workspace=llm_weight_workspace+'checkpoints/'+target_model
llm_finetune_savepath=llm_weight_workspace+'finetuned/'+target_model
#model_dir = snapshot_download(target_model, revision='v1.0.4',cache_dir=llm_weight_workspace)
import warnings
warnings.filterwarnings('ignore')
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,AutoConfig, AutoModel, BitsAndBytesConfig
from transformers.generation.utils import GenerationConfig
import torch.nn as nn
from torchkeras.chat import ChatLLM 

class ModelLoader(object):  # 创建Circle类
   def __init__(self, model_name_or_path):  # 约定成俗这里应该使用r，它与self.r中的r同名
       self.model_name_or_path = model_name_or_path
       self.bnb_config=BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                llm_int8_threshold=6.0,
                llm_int8_has_fp16_weight=False,
            )
   def load(self,for_training=True):  
    self.tokenizer = AutoTokenizer.from_pretrained(
       self.model_name_or_path, trust_remote_code=True)
    if for_training:
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name_or_path,
                    quantization_config=self.bnb_config, 
                    trust_remote_code=True)
    else:
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name_or_path,
                    quantization_config=self.bnb_config,
                    trust_remote_code=True)
    
    self.model.generation_config = GenerationConfig.from_pretrained(self.model_name_or_path)
    self.llm = ChatLLM(self.model,self.tokenizer)
    self.model.config.use_cache = False
    return self.model,self.tokenizer,self.llm
    
model_loader=ModelLoader(model_name_or_path=model_name_or_path)
model,tokenizer,llm=model_loader.load()


2023-10-19 11:12:39,727 - modelscope - INFO - PyTorch version 2.1.0 Found.
2023-10-19 11:12:39,731 - modelscope - INFO - Loading ast index from /home/ecs-assist-user/.cache/modelscope/ast_indexer
2023-10-19 11:12:39,899 - modelscope - INFO - Loading done! Current index file version is 1.9.2, with md5 fd57df25cf18fc0a972c98cfdffc3f42 and a total number of 941 components indexed
The model is automatically converting to fp16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.45 GiB. GPU 0 has a total capacty of 15.77 GiB of which 31.38 MiB is free. Process 6265 has 14.48 GiB memory in use. Including non-PyTorch memory, this process has 1.01 GiB memory in use. Of the allocated memory 122.86 MiB is allocated by PyTorch, and 155.14 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF